UNIFICACION ALQUILER


In [ ]:
import pandas as pd
from pathlib import Path
import re
import unicodedata

# Ruta alquiler
DATA_DIR = Path(r"C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\Alquiler")


def read_csv_safely(p: Path) -> pd.DataFrame:

    for enc in ("utf-8", "latin-1"):
        try:
            return pd.read_csv(p, encoding=enc)
        except Exception:
            pass
    for enc in ("utf-8", "latin-1"):
        try:
            return pd.read_csv(p, encoding=enc, sep=";")
        except Exception:
            pass
    raise RuntimeError(f"No pude leer {p}")

def parse_distrito_from_filename(p: Path) -> str:
    
    name = p.stem.lower()
    m = re.search(r"distrito_([a-z\-ñ]+)_alquiler", name)
    return m.group(1) if m else None

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    
    cols_map = {
        "precio":"precio",
        "localidad":"localidad",
        "tamaño":"tamanio",
        "tamano":"tamanio",
        "tamanio":"tamanio",
        "habitaciones":"habitaciones",
        "descripcion":"descripcion",
        "link":"link",
        "descripcion_larga":"descripcion_larga",
        "distrito":"distrito",
    }
    expected = ["precio","localidad","tamanio","habitaciones","descripcion","link","descripcion_larga","distrito"]
    df = df.copy()
    df.columns = [c.strip().lower() for c in df.columns]
    df = df.rename(columns={c: cols_map.get(c, c) for c in df.columns})
    for c in expected:
        if c not in df.columns:
            df[c] = pd.NA
    return df[expected]

def std_text(s: str) -> str:
    
    if pd.isna(s): return s
    s = str(s).lower().strip()
    s = ''.join(
        ch for ch in unicodedata.normalize('NFKD', s)
        if not unicodedata.combining(ch)
    )
    s = s.replace("-", "").replace(" ", "")
    return s

# Recorrer archivos y unir
files = sorted(DATA_DIR.glob("*.csv"))
frames = []

for p in files:
    df = read_csv_safely(p)
    df = normalize_columns(df)

    # Completar distrito desde el nombre si no viene dentro del CSV
    distrito_name = parse_distrito_from_filename(p)
    if df["distrito"].isna().all() or (df["distrito"].astype(str).str.strip() == "").all():
        df["distrito"] = distrito_name
    else:
        
        df["distrito"] = df["distrito"].astype(str).str.lower().str.strip()

    
    df["operacion"] = "alquiler"
    df["origen_archivo"] = p.name
    
    df["distrito_std"] = df["distrito"].apply(std_text)

    frames.append(df)

df_alq = pd.concat(frames, ignore_index=True)


if "link" in df_alq.columns:
    df_alq = df_alq.drop_duplicates(subset=["link","operacion"], keep="first")

# Guardar resultados
out_csv = DATA_DIR / "alquiler_unificado.csv"
out_parquet = DATA_DIR / "alquiler_unificado.parquet"

df_alq.to_csv(out_csv, index=False)
try:
    df_alq.to_parquet(out_parquet, index=False)
except Exception:
    pass  


print("Archivos guardados:\n -", out_csv, "\n -", out_parquet if out_parquet.exists() else "(parquet omitido)")
print("Filas:", len(df_alq), "| Columnas:", len(df_alq.columns))
print("Distritos detectados:", sorted(df_alq['distrito'].dropna().unique().tolist())[:10], "...")


✅ Listo.
Archivos guardados:
 - C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\Alquiler\alquiler_unificado.csv 
 - C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\Alquiler\alquiler_unificado.parquet
Filas: 6347 | Columnas: 11
Distritos detectados: ['arganzuela', 'barajas', 'barrio-de-salamanca', 'carabanchel', 'centro', 'chamartin', 'chamberi', 'ciudad-lineal', 'fuencarral', 'hortaleza'] ...


UNIFICACION VENTA

In [ ]:
import pandas as pd
from pathlib import Path
import re
import unicodedata

# Ruta venta
DATA_DIR = Path(r"C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\Venta")


def read_csv_safely(p: Path) -> pd.DataFrame:
    for enc in ("utf-8", "latin-1"):
        try:
            return pd.read_csv(p, encoding=enc)
        except Exception:
            pass
    for enc in ("utf-8", "latin-1"):
        try:
            return pd.read_csv(p, encoding=enc, sep=";")
        except Exception:
            pass
    raise RuntimeError(f"No pude leer {p}")

def parse_distrito_from_filename(p: Path) -> str:
    
    name = p.stem.lower()
    m = re.search(r"distrito_([a-z\-ñ]+)_venta", name)
    return m.group(1) if m else None

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols_map = {
        "precio":"precio",
        "localidad":"localidad",
        "tamaño":"tamanio",
        "tamano":"tamanio",
        "tamanio":"tamanio",
        "habitaciones":"habitaciones",
        "descripcion":"descripcion",
        "link":"link",
        "descripcion_larga":"descripcion_larga",
        "distrito":"distrito",
    }
    expected = ["precio","localidad","tamanio","habitaciones","descripcion","link","descripcion_larga","distrito"]
    df = df.copy()
    df.columns = [c.strip().lower() for c in df.columns]
    df = df.rename(columns={c: cols_map.get(c, c) for c in df.columns})
    for c in expected:
        if c not in df.columns:
            df[c] = pd.NA
    return df[expected]

def std_text(s: str) -> str:
    if pd.isna(s): return s
    s = str(s).lower().strip()
    s = ''.join(ch for ch in unicodedata.normalize('NFKD', s) if not unicodedata.combining(ch))
    return s.replace("-", "").replace(" ", "")

# Unificación
files = sorted(DATA_DIR.glob("*.csv"))
frames = []

for p in files:
    df = read_csv_safely(p)
    df = normalize_columns(df)

    distrito_name = parse_distrito_from_filename(p)
    if df["distrito"].isna().all() or (df["distrito"].astype(str).str.strip() == "").all():
        df["distrito"] = distrito_name
    else:
        df["distrito"] = df["distrito"].astype(str).str.lower().str.strip()

    df["operacion"] = "venta"
    df["origen_archivo"] = p.name
    df["distrito_std"] = df["distrito"].apply(std_text)

    frames.append(df)

df_venta = pd.concat(frames, ignore_index=True)


if "link" in df_venta.columns:
    df_venta = df_venta.drop_duplicates(subset=["link","operacion"], keep="first")

# Guardar
out_csv = DATA_DIR / "venta_unificado.csv"
out_parquet = DATA_DIR / "venta_unificado.parquet"

df_venta.to_csv(out_csv, index=False)
try:
    df_venta.to_parquet(out_parquet, index=False) 
except Exception:
    pass


print("Guardado:", out_csv, "| filas:", len(df_venta), "columnas:", len(df_venta.columns))
print("Ejemplo de distritos:", sorted(df_venta['distrito'].dropna().unique().tolist())[:10], "…")


✅ Listo
Guardado: C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\Venta\venta_unificado.csv | filas: 9613 columnas: 11
Ejemplo de distritos: ['arganzuela', 'barajas', 'barrio-de-salamanca', 'carabanchel', 'centro', 'chamartin', 'chamberi', 'ciudad-lineal', 'fuencarral', 'hortaleza'] …


UNIFICACION TOTAL DE CSV

In [ ]:
import pandas as pd
from pathlib import Path

# Rutas de datasets
BASE_DIR = Path(r"C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas")
f_alq = BASE_DIR / "Alquiler" / "alquiler_unificado.csv"
f_vta = BASE_DIR / "Venta" / "venta_unificado.csv"


alq = pd.read_csv(f_alq)
vta = pd.read_csv(f_vta)

# Alinear columnas
all_cols = sorted(set(alq.columns) | set(vta.columns))
for df in (alq, vta):
    for c in all_cols:
        if c not in df.columns:
            df[c] = pd.NA
    df = df[all_cols]

# Unir
df_all = pd.concat([alq[all_cols], vta[all_cols]], ignore_index=True)


if {"link","operacion"}.issubset(df_all.columns):
    df_all = df_all.drop_duplicates(subset=["link","operacion"], keep="first")

#Guardar resultado
out_csv = BASE_DIR / "inmuebles_unificado_total.csv"
out_parquet = BASE_DIR / "inmuebles_unificado_total.parquet"

df_all.to_csv(out_csv, index=False)
try:
    df_all.to_parquet(out_parquet, index=False)
except Exception:
    pass


print("Archivo:", out_csv, "| Filas:", len(df_all), "| Columnas:", len(df_all.columns))
print("Operaciones:", df_all["operacion"].value_counts(dropna=False).to_dict())
print("Ejemplo columnas:", df_all.columns.tolist()[:12], "…")


✅ Unificación completa
Archivo: C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\inmuebles_unificado_total.csv | Filas: 15960 | Columnas: 11
Operaciones: {'venta': 9613, 'alquiler': 6347}
Ejemplo columnas: ['descripcion', 'descripcion_larga', 'distrito', 'distrito_std', 'habitaciones', 'link', 'localidad', 'operacion', 'origen_archivo', 'precio', 'tamanio'] …


CREAR LOS EMBEDDINGS

In [4]:
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


EMBEDDINGS

In [ ]:
import pandas as pd
import re
import unicodedata
from datetime import datetime
from pathlib import Path
from sentence_transformers import SentenceTransformer

# Archivo de entrada/salida
INPUT_FILE = Path(r"C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\inmuebles_unificado_total.csv")
OUTPUT_BASENAME = INPUT_FILE.with_suffix('').name  

# Cargar 
df = pd.read_csv(INPUT_FILE)


def std_text(s: str) -> str:
    if pd.isna(s): return s
    s = str(s).lower().strip()
    s = ''.join(ch for ch in unicodedata.normalize('NFKD', s) if not unicodedata.combining(ch))
    return s


KEYWORDS = ["antigüedad", "baños", "garaje", "trastero", "piscina", "terraza"]
for kw in KEYWORDS:
    col = f"has_{kw}".replace("ñ", "n")  # ej: has_banos
    df[col] = df["descripcion_larga"].fillna("").str.lower().apply(lambda x: 1 if kw in x else 0)


def precio_to_float(s):
    if pd.isna(s): return pd.NA
    s = str(s).replace(".", "").replace("€","").replace(" ", "").replace(",", ".")
    try: return float(s)
    except: return pd.NA
df["precio_num"] = df["precio"].apply(precio_to_float)


df["distrito_std"] = df["distrito"].astype(str).map(std_text).str.replace("-", "").str.replace(" ", "")

# Extracciones numéricas (baños, año construcción, antigüedad)
MAPA_NUM = {"un":1,"una":1,"uno":1,"dos":2,"tres":3,"cuatro":4,"cinco":5,"seis":6,"siete":7,"ocho":8,"nueve":9,"diez":10}
CURRENT_YEAR = datetime.now().year

def extract_num_banos(texto: str):
    if pd.isna(texto): return None
    t = texto.lower()
    m = re.search(r'(\d+)\s*bañ', t)
    if m: return int(m.group(1))
    for palabra, val in MAPA_NUM.items():
        if re.search(rf'\b{palabra}\s+bañ', t): return val
    return None

def extract_year_built(texto: str):
    if pd.isna(texto): return None
    t = texto.lower()
    patrones = [
        r'construid[oa]\s+en\s+(19\d{2}|20\d{2})',
        r'año\s*(de)?\s*construcci[oó]n[:\s]+(19\d{2}|20\d{2})',
        r'del?\s+año\s+(19\d{2}|20\d{2})',
        r'en\s+(19\d{2}|20\d{2})\s+se\s+construy[oó]',
        r'obra\s+del?\s+(19\d{2}|20\d{2})',
        r'edificio\s+(del?\s+)?(19\d{2}|20\d{2})'
    ]
    for pat in patrones:
        m = re.search(pat, t)
        if m:
            y = int(m.groups()[-1])
            if 1900 <= y <= CURRENT_YEAR: return y
    m = re.search(r'\b(19\d{2}|20\d{2})\b', t)
    if m:
        y = int(m.group(1))
        if 1900 <= y <= CURRENT_YEAR: return y
    return None

def extract_antiguedad(texto: str):
    if pd.isna(texto): return None
    t = texto.lower()
    if re.search(r'\b(obra\s+nueva|a\s+estrenar)\b', t): return 0
    m = re.search(r'(\d+)\s*años?\s*(de\s*)?antig(ü|u)edad', t)
    if m: return int(m.group(1))
    for palabra, val in MAPA_NUM.items():
        if re.search(rf'{palabra}\s+años?\s*(de\s*)?antig(ü|u)edad', t): return val
    return None

df["num_banos"] = df["descripcion_larga"].apply(extract_num_banos)
df["anio_construccion"] = df["descripcion_larga"].apply(extract_year_built)
df["antiguedad_texto"] = df["descripcion_larga"].apply(extract_antiguedad)
df["antiguedad_calc"] = df["anio_construccion"].apply(lambda y: CURRENT_YEAR - y if pd.notna(y) else None)
df["antiguedad_final"] = df["antiguedad_texto"].fillna(df["antiguedad_calc"])

# Texto para el embedding
def build_embedding_text(row):
    partes = [
        f"operacion: {row.get('operacion','')}",
        f"distrito: {row.get('distrito','')}",
        f"tamanio_m2: {row.get('tamanio','')}",
        f"habitaciones: {row.get('habitaciones','')}",
        f"banos: {row.get('num_banos','')}",
        f"garaje: {row.get('has_garaje',0)}",
        f"trastero: {row.get('has_trastero',0)}",
        f"piscina: {row.get('has_piscina',0)}",
        f"terraza: {row.get('has_terraza',0)}",
        f"antiguedad: {row.get('antiguedad_final','')}",
        f"descripcion: {str(row.get('descripcion_larga',''))[:1200]}",
    ]
    return " | ".join(map(lambda s: str(s), partes))

df["texto_embedding"] = df.apply(build_embedding_text, axis=1)

# Generar embeddings 
model = SentenceTransformer("all-MiniLM-L6-v2")  # rápido y sólido
df["embedding"] = df["texto_embedding"].fillna("").apply(lambda x: model.encode(x))


cols = [c for c in df.columns if c != "embedding"] + ["embedding"]
df = df[cols]

OUT_DIR = INPUT_FILE.parent
df.to_pickle(OUT_DIR / f"{OUTPUT_BASENAME}_with_embeddings.pkl")

# CSV 
df_csv = df.copy()
df_csv["embedding_str"] = df_csv["embedding"].apply(lambda v: ",".join(map(str, v)))
df_csv = df_csv.drop(columns=["embedding"])
df_csv.to_csv(OUT_DIR / f"{OUTPUT_BASENAME}_with_embeddings.csv", index=False)

# Parquet 
try:
    df.to_parquet(OUT_DIR / f"{OUTPUT_BASENAME}_with_embeddings.parquet", index=False)
except Exception:
    pass

print("Embeddings generados y guardados en:", OUT_DIR)



Embeddings generados y guardados en: C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas


VERIFICACION

In [ ]:
import pandas as pd

df_pkl = pd.read_pickle(r"C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\inmuebles_unificado_total_with_embeddings.pkl"
)
print(df_pkl.head(3))
print("\nColumnas:", df_pkl.columns.tolist())


                       descripcion  \
0  Planta 3ª exterior con ascensor   
1  Planta 3ª exterior con ascensor   
2  Planta 6ª exterior con ascensor   

                                   descripcion_larga    distrito distrito_std  \
0  \nDISPONIBLE A PARTIR 1 DE DICIEMBRE 2025\nAla...  arganzuela   arganzuela   
1  \nDISPONIBLE EN OCTUBRE! MODUS HOME PONE A TU ...  arganzuela   arganzuela   
2  \nPrecioso apartamento, reformado íntegramente...  arganzuela   arganzuela   

   habitaciones                  link  \
0             1  /inmueble/109262818/   
1             2  /inmueble/109227606/   
2             1   /inmueble/25140719/   

                                           localidad operacion  \
0             Piso en Calle de Ricardo Damas Legazpi  alquiler   
1  Dúplex en Calle de Bernardino Obregón Palos de...  alquiler   
2                   Piso en Paseo de las Delicias 13  alquiler   

           origen_archivo precio  ... has_piscina  has_terraza  precio_num  \
0  alquiler_un

In [14]:
print("Filas:", df_pkl.shape[0], "| Columnas:", df_pkl.shape[1])


Filas: 15960 | Columnas: 25


In [15]:

print(type(df_pkl["embedding"].iloc[0]))
print("Longitud embedding:", len(df_pkl["embedding"].iloc[0]))
print("Primeros 10 valores:", df_pkl["embedding"].iloc[0][:10])


<class 'numpy.ndarray'>
Longitud embedding: 384
Primeros 10 valores: [ 0.07376748  0.01204776  0.00297735 -0.04105594 -0.10310872 -0.00557283
  0.04342492 -0.01798573 -0.08226091 -0.0104314 ]


In [ ]:
print(df_pkl[[
    "precio", "precio_num", "num_banos", 
    "anio_construccion", "antiguedad_final", 
    "has_garaje", "has_piscina", "has_terraza"
]].head(10))


  precio  precio_num  num_banos  anio_construccion  antiguedad_final  \
0   1.28       128.0        NaN             2025.0               0.0   
1    1.7        17.0        2.0                NaN               NaN   
2    1.3        13.0        NaN                NaN               NaN   
3    1.5        15.0        NaN                NaN               NaN   
4    1.0        10.0        1.0                NaN               NaN   
5    1.8        18.0        NaN                NaN               NaN   
6    1.3        13.0        NaN                NaN               NaN   
7   1.35       135.0        NaN                NaN               NaN   
8    1.6        16.0        NaN                NaN               NaN   
9  1.423      1423.0        NaN                NaN               NaN   

   has_garaje  has_piscina  has_terraza  
0           0            0            0  
1           0            0            1  
2           0            0            0  
3           0            0            0

In [19]:
print("Operaciones disponibles:", df_pkl["operacion"].unique())
print("Distritos:", df_pkl["distrito"].unique()[:21])


Operaciones disponibles: ['alquiler' 'venta']
Distritos: ['arganzuela' 'barajas' 'barrio-de-salamanca' 'carabanchel' 'centro'
 'chamartin' 'chamberi' 'ciudad-lineal' 'fuencarral' 'hortaleza' 'latina'
 'moncloa' 'moratalaz' 'puente-de-vallecas' 'retiro' 'san-blas' 'tetuan'
 'usera' 'vicalvaro' 'villa-de-vallecas' 'villaverde']


In [21]:
print(df_pkl.columns.tolist())


['descripcion', 'descripcion_larga', 'distrito', 'distrito_std', 'habitaciones', 'link', 'localidad', 'operacion', 'origen_archivo', 'precio', 'tamanio', 'has_antigüedad', 'has_banos', 'has_garaje', 'has_trastero', 'has_piscina', 'has_terraza', 'precio_num', 'num_banos', 'anio_construccion', 'antiguedad_texto', 'antiguedad_calc', 'antiguedad_final', 'texto_embedding', 'embedding']


In [22]:
print(df_pkl.columns[:10])   # primeras 10 columnas


Index(['descripcion', 'descripcion_larga', 'distrito', 'distrito_std',
       'habitaciones', 'link', 'localidad', 'operacion', 'origen_archivo',
       'precio'],
      dtype='object')


CORRECCION DE LA COLUMNA PRECIO_NUM

In [27]:
def corregir_precio(s):
    if pd.isna(s): 
        return pd.NA
    s = str(s).replace("€","").replace(" ", "")
    
    # Caso con punto decimal y pocos dígitos 
    if re.match(r'^\d+(\.\d+)?$', s):
        valor = float(s)
        return round(valor * 1000)  
    
    # Caso con miles separadores 
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except:
        return pd.NA

df["precio_num"] = df["precio"].apply(corregir_precio)


In [28]:
# Mostrar ejemplos de la conversión
print(df[["precio", "precio_num"]].head(20))

# Revisar valores mínimos y máximos
print("\nEstadísticas precio_num:")
print(df["precio_num"].describe())

# Buscar precios pequeños sospechosos (ej: < 10,000 €)
print("\nPosibles errores (precio_num < 10000):")
print(df.loc[df["precio_num"] < 10000, ["precio", "precio_num"]].head(15))


   precio  precio_num
0    1.28      1280.0
1     1.7      1700.0
2     1.3      1300.0
3     1.5      1500.0
4     1.0      1000.0
5     1.8      1800.0
6     1.3      1300.0
7    1.35      1350.0
8     1.6      1600.0
9   1.423      1423.0
10    1.8      1800.0
11    1.6      1600.0
12    2.3      2300.0
13    2.0      2000.0
14    1.7      1700.0
15    1.5      1500.0
16   1.39      1390.0
17  1.499      1499.0
18    1.2      1200.0
19  1.636      1636.0

Estadísticas precio_num:
count    1.596000e+04
mean     5.502592e+05
std      1.015089e+06
min      1.000000e+03
25%      2.300000e+03
50%      2.515355e+05
75%      6.705000e+05
max      1.500000e+07
Name: precio_num, dtype: float64

Posibles errores (precio_num < 10000):
   precio  precio_num
0    1.28      1280.0
1     1.7      1700.0
2     1.3      1300.0
3     1.5      1500.0
4     1.0      1000.0
5     1.8      1800.0
6     1.3      1300.0
7    1.35      1350.0
8     1.6      1600.0
9   1.423      1423.0
10    1.8      1800.0

GUARDAR ARCHIVOS FINALES

In [ ]:
from pathlib import Path

# Carpeta de salida 
OUT_DIR = Path(r"C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas")

# Nombres de salida
out_csv = OUT_DIR / "inmuebles_unificado_total_final.csv"
out_parquet = OUT_DIR / "inmuebles_unificado_total_final.parquet"
out_pkl = OUT_DIR / "inmuebles_unificado_total_final.pkl"

#Guardar en CSV 
df_csv = df.copy()
if "embedding" in df_csv.columns:
    df_csv["embedding_str"] = df_csv["embedding"].apply(lambda v: ",".join(map(str, v)))
    df_csv = df_csv.drop(columns=["embedding"])
df_csv.to_csv(out_csv, index=False)

# Guardar en Parquet 
try:
    df.to_parquet(out_parquet, index=False)
except Exception as e:
    print("No se pudo guardar en Parquet:", e)

# Guardar en PKL 
df.to_pickle(out_pkl)


print(" - CSV:", out_csv)
print(" - PKL:", out_pkl)
print(" - Parquet:", out_parquet)


✅ Archivos guardados:
 - CSV: C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\inmuebles_unificado_total_final.csv
 - PKL: C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\inmuebles_unificado_total_final.pkl
 - Parquet: C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\inmuebles_unificado_total_final.parquet


ANALISIS INICIAL

In [ ]:
import pandas as pd

# Cargar el archivo final 
df = pd.read_pickle(r"C:\Users\andre\OneDrive\Documentos\IMF\TFM\Nuevos CSV\descargas\inmuebles_unificado_total_final.pkl")


In [ ]:
# Estructura del DataFrame

print(" Dimensiones del dataset:", df.shape)
print("\n Tipos de datos:")
print(df.dtypes)

🔹 Dimensiones del dataset: (15960, 25)

🔹 Tipos de datos:
descripcion           object
descripcion_larga     object
distrito              object
distrito_std          object
habitaciones           int64
link                  object
localidad             object
operacion             object
origen_archivo        object
precio                object
tamanio               object
has_antigüedad         int64
has_banos              int64
has_garaje             int64
has_trastero           int64
has_piscina            int64
has_terraza            int64
precio_num           float64
num_banos            float64
anio_construccion    float64
antiguedad_texto     float64
antiguedad_calc      float64
antiguedad_final     float64
texto_embedding       object
embedding             object
dtype: object


In [ ]:
# Valores nulos por columna

print("\n Porcentaje de nulos por columna:")
print((df.isna().mean() * 100).round(2).sort_values(ascending=False))


🔹 Porcentaje de nulos por columna:
antiguedad_calc      92.03
anio_construccion    92.03
antiguedad_texto     91.69
antiguedad_final     84.45
num_banos            80.03
descripcion           8.55
tamanio               0.90
descripcion_larga     0.47
has_trastero          0.00
texto_embedding       0.00
precio_num            0.00
has_terraza           0.00
has_piscina           0.00
has_banos             0.00
has_garaje            0.00
has_antigüedad        0.00
precio                0.00
origen_archivo        0.00
operacion             0.00
localidad             0.00
link                  0.00
habitaciones          0.00
distrito_std          0.00
distrito              0.00
embedding             0.00
dtype: float64


In [ ]:
# Descripción numérica

print("\n Estadísticas de columnas numéricas:")
print(df.describe().T)


🔹 Estadísticas de columnas numéricas:
                     count           mean           std     min      25%  \
habitaciones       15960.0       4.734023  1.238237e+01     1.0     2.00   
has_antigüedad     15960.0       0.001003  3.164750e-02     0.0     0.00   
has_banos          15960.0       0.110965  3.140985e-01     0.0     0.00   
has_garaje         15960.0       0.075877  2.648098e-01     0.0     0.00   
has_trastero       15960.0       0.054135  2.262917e-01     0.0     0.00   
has_piscina        15960.0       0.067419  2.507533e-01     0.0     0.00   
has_terraza        15960.0       0.160213  3.668150e-01     0.0     0.00   
precio_num         15960.0  550259.186779  1.015089e+06  1000.0  2300.00   
num_banos           3187.0       1.657358  7.914791e-01     1.0     1.00   
anio_construccion   1272.0    1994.226415  3.862009e+01  1900.0  1968.75   
antiguedad_texto    1327.0       0.026375  6.417341e-01     0.0     0.00   
antiguedad_calc     1272.0      30.773585  3.8620

In [35]:
# Columnas categóricas

print("\n Distribución de operaciones:")
print(df["operacion"].value_counts(dropna=False))

print("\n Ejemplo de distritos:")
print(df["distrito"].value_counts(dropna=False).head(10))


 Distribución de operaciones:
operacion
venta       9613
alquiler    6347
Name: count, dtype: int64

 Ejemplo de distritos:
distrito
chamartin              1157
tetuan                 1146
chamberi               1123
barrio-de-salamanca    1112
centro                 1098
retiro                 1064
moncloa                1027
arganzuela              970
ciudad-lineal           912
fuencarral              836
Name: count, dtype: int64


In [39]:
# Variables de interés extra

cols_extra = ["num_banos", "anio_construccion", "antiguedad_final", "has_garaje", "has_trastero", "has_piscina", "has_terraza"]
print("\n Resumen de variables extra:")
print(df[cols_extra].describe(include="all").T)


 Resumen de variables extra:
                     count         mean        std     min      25%     50%  \
num_banos           3187.0     1.657358   0.791479     1.0     1.00     2.0   
anio_construccion   1272.0  1994.226415  38.620089  1900.0  1968.75  2016.0   
antiguedad_final    2481.0    14.129786  30.037317     0.0     0.00     0.0   
has_garaje         15960.0     0.075877   0.264810     0.0     0.00     0.0   
has_trastero       15960.0     0.054135   0.226292     0.0     0.00     0.0   
has_piscina        15960.0     0.067419   0.250753     0.0     0.00     0.0   
has_terraza        15960.0     0.160213   0.366815     0.0     0.00     0.0   

                      75%     max  
num_banos             2.0    10.0  
anio_construccion  2025.0  2025.0  
antiguedad_final      7.0   125.0  
has_garaje            0.0     1.0  
has_trastero          0.0     1.0  
has_piscina           0.0     1.0  
has_terraza           0.0     1.0  
